In [27]:
import sys

from from_root import from_root
from pathlib import Path

In [28]:
sys.path.insert(0, str(from_root("src")))

from read_and_write_docs import read_completed_excel_result
from excel_functions import create_excel_template

## Convert Single Document

In [29]:
excel_path = '/Volumes/BCross/paraphrase examples/Wiki-test-gpt-4-1-v2/hootmag_text_12 vs hootmag_text_13.xlsx'

In [30]:
data = read_completed_excel_result(excel_path)

In [31]:
# create_excel_template(
#     known=data['known'].drop(columns=['include_phrase']),
#     unknown=data['unknown'].drop(columns=['include_phrase']),
#     no_context=data['no_context'],
#     metadata=data['metadata'],
#     docs=data['docs'],
#     path=excel_path,
#     known_sheet = "known",
#     unknown_sheet = "unknown",
#     nc_sheet = "no context",
#     metadata_sheet = "metadata",
#     docs_sheet = "docs",
#     llr_sheet = "LLR",
#     use_xlookup = False
# )

## Convert Multiple Files

In [32]:
save_dir = Path('/Volumes/BCross/paraphrase examples/Wiki-test-gpt-4-1-v2')

for excel_path in sorted(save_dir.glob("*.xlsx")):
    if excel_path.name.startswith("~$"):
        continue  # skip Excel temp/lock files


    print(f"Processing: {excel_path.name}")
    
    data = read_completed_excel_result(excel_path)
    
    create_excel_template(
        known=data['known'].drop(columns=['include_phrase']),
        unknown=data['unknown'].drop(columns=['include_phrase']),
        no_context=data['no_context'],
        metadata=data['metadata'],
        docs=data['docs'],
        path='/Volumes/BCross/paraphrase examples/Wiki-test-gpt-4-1-v2/hootmag_text_12 vs hootmag_text_13.xlsx',
        known_sheet = "known",
        unknown_sheet = "unknown",
        nc_sheet = "no context",
        metadata_sheet = "metadata",
        docs_sheet = "docs",
        llr_sheet = "LLR",
        use_xlookup = False
    )

    print(f"  ✓ Wrote: {excel_path}")

Processing: hootmag_text_12 vs hootmag_text_13.xlsx
  ✓ Wrote: /Volumes/BCross/paraphrase examples/Wiki-test-gpt-4-1-v2/hootmag_text_12 vs hootmag_text_13.xlsx
Processing: icarus3_text_1 vs icarus3_text_4.xlsx
  ✓ Wrote: /Volumes/BCross/paraphrase examples/Wiki-test-gpt-4-1-v2/icarus3_text_1 vs icarus3_text_4.xlsx
Processing: icarus3_text_2 vs icarus3_text_4.xlsx
  ✓ Wrote: /Volumes/BCross/paraphrase examples/Wiki-test-gpt-4-1-v2/icarus3_text_2 vs icarus3_text_4.xlsx
Processing: icarus3_text_3 vs icarus3_text_4.xlsx
  ✓ Wrote: /Volumes/BCross/paraphrase examples/Wiki-test-gpt-4-1-v2/icarus3_text_3 vs icarus3_text_4.xlsx
Processing: ivoshandor_text_2 vs jasper_deng_text_4.xlsx
  ✓ Wrote: /Volumes/BCross/paraphrase examples/Wiki-test-gpt-4-1-v2/ivoshandor_text_2 vs jasper_deng_text_4.xlsx
Processing: ivoshandor_text_4 vs jasper_deng_text_4.xlsx
  ✓ Wrote: /Volumes/BCross/paraphrase examples/Wiki-test-gpt-4-1-v2/ivoshandor_text_4 vs jasper_deng_text_4.xlsx
Processing: ivoshandor_text_5 vs